In [1]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Downloading ollama...
######################################################################## 100.0%-=O=- #   #   #    #                                                                                   21.1%         27.2%
>>> Installing ollama to /usr/local/bin...
>>> Creating ollama user...
>>> Adding ollama user to render group...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [60]:
import crewai, crewai_tools
from langchain_community.llms import Ollama
from IPython.display import Markdown

In [61]:
from crewai import Agent, Task, Crew, Process

llm = Ollama(model='llama3')

nurse = Agent(
    role="A hospital nurse",
    goal="Your goal is to collect patient information and symptoms. You ask questions directly to the patient when needed. ",
    backstory="You are a nurse working at a hospital recieving emergency room patients. ",
    verbose=True,
    allow_delegation=False,
    llm=llm
)

doctor = Agent(
    role="A hospital doctor",
    goal="Your goal is to diagnose the patient and identify the disease that they might be suffering from. "
    "You also have to provide the next course of action for the patient. "
    "You also might need to call in a specialist (for example: a dermatologist for skin diseases) if one is available. ",
    backstory="You are a doctor working at a hospital. You have to diagnose patients' diseases given the nurse's reports.",
    verbose = True,
    allow_delegaton=False,
    llm=llm
)

In [62]:
collect_info = Task(
    description=("Given the patients input: {input}. You will collect patient information. You will return a question for further precision if needed."),
    expected_output="A report of the patient's symptoms from most severe to least severe, keep it simple and close to the patient's inputs. "
    "Ask a question to the patient if more information from them is needed. ",
    human_input=True,

    agent=nurse
)

diagnose = Task(
    description=("Given the nurse's reported information from the patient. You must make an assessment of the results. "),
    expected_output="You must provide a diagnosis or potential diagnoses. "
    "You must determine whether more information is needed from the patient and the next course of action. "
    "Tell the nurse whether more information is needed, provide questions for the patient, and consider the task done. "
    "If no specialists are available, try to come up with an answer. ",
    agent=doctor
)

In [63]:
crew = Crew(
    agents=[nurse, doctor],
    tasks=[collect_info, diagnose],
    verbose=True
)

2024-08-14 17:49:12,150 - 140263783333888 - __init__.py-__init__:531 - WARNING: Overriding of current TracerProvider is not allowed


In [64]:
Markdown(crew.kickoff(inputs={'input': "I have shortness of breath"}).raw)

 [2024-08-14 17:49:13][DEBUG]: == Working Agent: A hospital nurse
 [2024-08-14 17:49:13][INFO]: == Starting Task: Given the patients input: I have shortness of breath. You will collect patient information. You will return a question for further precision if needed.


> Entering new CrewAgentExecutor chain...


This is the agent's final answer: Patient Symptoms (from most severe to least severe):
Shortness of breath

Additional Information Needed:

Can you describe what's happening when you experience shortness of breath? Is it sudden or does it come on gradually?
Please provide feedback:  gradually


Thought: I now have all the necessary information from the patient


Action: Compile the patient's symptoms into a report

Final Answer:
Patient Symptoms (from most severe to least severe):
Shortness of breath

Additional Information Needed:

Can you describe what's happening when you experience shortness of breath? Is it sudden or does it come on gradually

> Finished chain.
 [2024-08-14 17:49:41][DEBUG]: == [A hospital nurse] Task output: Patient Symptoms (from most severe to least severe):
Shortness of breath

Additional Information Needed:

Can you describe what's happening when you experience shortness of breath? Is it sudden or does it come on gradually


 [2024-08-14 17:49:41][DEBUG]: == Working Agent: A hospital doctor
 [2024-08-14 17:49:41][INFO]: == Starting Task: Given the nurse's reported information from the patient. You must make an assessment of the results. 


> Entering new CrewAgentExecutor chain...
Thought: Let's start by asking some questions to gather more informat

KeyboardInterrupt: 

In [ ]:
from crewai_tools import BaseTool

class PromptingPatientTool():
    name: str = "Patient Prompting"
    description: str = ""